In [1]:
import os
from rdflib import Graph, Namespace, RDF, RDFS, XSD

# Create a Graph
EX = Namespace("http://example.org/research_paper#")

# Create a graph
g = Graph()

# Bind common prefixes (for readability)
g.bind("ex", EX)
g.bind("rdfs", RDFS)
g.bind("rdf", RDF)

In [ ]:
def create_property(g, namespace, property, domain, range):
    g.add((namespace[property], RDF.type, RDF.Property))
    g.add((namespace[property], RDFS.domain, namespace[domain]))
    g.add((namespace[property], RDFS.range, namespace[range]))

def create_class(g, namespace, class_name):
    g.add((namespace[class_name], RDF.type, RDFS.Class))

def create_subclass(g, namespace, subclass, superclass):
    g.add((namespace[subclass], RDFS.subClassOf, namespace[superclass]))



In [4]:
# Create classes
create_class(g, EX, "Paper")
create_class(g, EX, "Author")
create_class(g, EX, "Journal")
create_class(g, EX, "Conference")
create_class(g, EX, "Topic")
create_class(g, EX, "Abstract")
create_class(g, EX, "Review")
create_class(g, EX, "Publisher")
create_class(g, EX, "Edition")
create_class(g, EX, "Volume")

In [5]:
# Restrictions on classes
create_subclass(g, EX, "Conference", "Publisher")
create_subclass(g, EX, "Journal", "Publisher")

In [8]:
# Create properties
create_property(g, EX, "written_by", "Paper", "Author")
create_property(g, EX, "cites", "Paper", "Paper")
create_property(g, EX, "related_to", "Paper", "Topic")
create_property(g, EX, "corresponds", "Paper", "Author")
create_property(g, EX, "has_review", "Paper", "Review")
create_property(g, EX, "is_reviewer", "Author", "Review")
create_property(g, EX, "published_in", "Paper", "Publisher")
create_property(g, EX, "has_edition", "Conference", "Edition")
create_property(g, EX, "has_volume", "Journal", "Volume")

g.add((EX.has_number, RDF.type, RDF.Property))
g.add((EX.has_number, RDFS.domain, EX.Volume))
g.add((EX.has_number, RDFS.range, XSD.string))
g.add((EX.has_date, RDF.type, RDF.Property))
g.add((EX.has_date, RDFS.domain, EX.Edition))
g.add((EX.has_date, RDFS.range, XSD.string))
g.add((EX.has_date, RDF.type, RDF.Property))
g.add((EX.has_city, RDFS.domain, EX.Edition))
g.add((EX.has_city, RDFS.range, XSD.string))
g.add((EX.has_abstract, RDF.type, RDF.Property))
g.add((EX.has_abstract, RDFS.domain, EX.Paper))
g.add((EX.has_abstract, RDFS.range, XSD.string))

<Graph identifier=N350d3d42015849e5be1ad7ab8d7a3c0b (<class 'rdflib.graph.Graph'>)>

In [9]:
for s, p, o in g:
    print((s, p, o))

(rdflib.term.URIRef('http://example.org/research_paper#Journal'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))
(rdflib.term.URIRef('http://example.org/research_paper#has_number'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), rdflib.term.URIRef('http://example.org/research_paper#Volume'))
(rdflib.term.URIRef('http://example.org/research_paper#has_review'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'))
(rdflib.term.URIRef('http://example.org/research_paper#has_city'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'), rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))
(rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#has_date'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-

In [11]:
import os

# Use current working directory instead of script path
script_dir = os.getcwd()  # Works in Jupyter
rdf_file_path = os.path.join(script_dir, "example_graph.ttl")

# Save the graph
g.serialize(destination=rdf_file_path, format="turtle")

print(f"RDF saved to: {rdf_file_path}")

RDF saved to: c:\Users\spiss\Desktop\Programmi\GitHub\SDM_lab2\example_graph.ttl
